
Use Pandas to do some feature engineering.

Transform categorical variables into binary variables.

Write a function to compute the number of misclassified examples in an intermediate node.

Write a function to find the best feature to split on.

Build a binary decision tree from scratch.

Make predictions using the decision tree.

Evaluate the accuracy of the decision tree.

Visualize the decision at the root node.

Important Note: In this assignment, we will focus on building decision trees where the data contain only binary (0 or 1) features. This allows us to avoid dealing with:

    Multiple intermediate nodes in a split
    
    The thresholding issues of real-valued features.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
loan_df = pd.read_csv('data/lending-club-data.csv',low_memory=False)

In [5]:
loan_df.head(1).T

,0
id,1077501
member_id,1296599
loan_amnt,5000
funded_amnt,5000
funded_amnt_inv,4975
...,...
payment_inc_ratio,8.1435
final_d,20141201T000000
last_delinq_none,1
last_record_none,1


In [6]:
#Labels  we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.
#safe_loans = 1 => safe
#safe_loans = -1 => risky
loan_df['safe_loans'] = loan_df['bad_loans'].apply(lambda x : +1 if x ==0 else -1)
loan_df.drop(columns=['bad_loans'])

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none,safe_loans
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,1.0,1.0,1.0,0,8.143500,20141201T000000,1,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,1.0,1.0,1.0,1,2.393200,20161201T000000,1,1,1,-1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,0,8.259550,20141201T000000,1,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,1.0,1.0,1.0,0,8.275850,20141201T000000,0,1,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,1.0,1.0,1.0,0,5.215330,20141201T000000,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122602,9856168,11708132,6000,6000,6000,60 months,23.40,170.53,E,E5,...,0.0,1.0,1.0,1,4.487630,20190101T000000,0,1,0,-1
122603,9795013,11647121,15250,15250,15250,36 months,17.57,548.05,D,D2,...,0.0,0.0,1.0,0,10.117800,20170101T000000,0,0,0,1
122604,9695736,11547808,8525,8525,8525,60 months,18.25,217.65,D,D3,...,0.0,1.0,1.0,0,6.958120,20190101T000000,0,1,0,-1
122605,9684700,11536848,22000,22000,22000,60 months,19.97,582.50,D,D5,...,1.0,0.0,1.0,0,8.961540,20190101T000000,1,0,1,-1


In [7]:
loan_df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra



in this assignment, we will just be using 4 categorical features:

grade of the loan

the length of the loan term

the home ownership status: own, mortgage, rent

number of years of employment.

Since we are building a binary decision tree, we will have to convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.


In [8]:
features = [
    'grade',
    'term',
    'home_ownership',
    'emp_length',    
]
target = 'safe_loans'
loan_df = loan_df[features + [target]]
loan_df.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


In [12]:
print(sum(1 for x in loan_df[target].values if x==-1))
print(sum(1 for x in loan_df[target].values if x==1))

23150
99457



Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We use seed=1 so everyone gets the same results.
